In [ ]:
%load_ext autoreload
%autoreload 2

import random
import pickle
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, f1_score
from ray import tune
from pathlib import Path
from tqdm.notebook import tqdm

from utils import initialize_seeds
from data_utils import HateDataset, get_results

In [ ]:
DEV = False

In [ ]:
initialize_seeds()

In [ ]:
data_path = Path("./hatecheck-data")

In [ ]:
hatecheck_df = pd.read_csv(data_path/"test_suite_cases.csv", index_col=0)

In [ ]:
hatecheck_df.label_gold.replace({'hateful': 1, 'non-hateful': 0}, inplace=True)

## Stratified sampling with held-out rules and groups

In [ ]:
hatecheck_df.groupby("label_gold")["functionality"].value_counts()

In [ ]:
heldout_nonhate_funcs = random.sample(pd.unique(hatecheck_df[hatecheck_df["label_gold"]==0]["functionality"]).tolist(),2); heldout_nonhate_funcs

In [ ]:
heldout_hate_funcs = random.sample(pd.unique(hatecheck_df[hatecheck_df["label_gold"]==1]["functionality"]).tolist(),3); heldout_hate_funcs

In [ ]:
heldout_idents = random.sample(pd.unique(hatecheck_df["target_ident"].dropna()).tolist(),2); heldout_idents

In [ ]:
held_out_dic = {}
held_out_dic["hateful_funcs"] = heldout_hate_funcs
held_out_dic["nonhateful_funcs"] = heldout_nonhate_funcs
held_out_dic["idents"] = heldout_idents

In [ ]:
with open("./data/held_out_dic.pkl", "wb") as file:
    pickle.dump(held_out_dic, file)

In [ ]:
hatecheck_seen = hatecheck_df[~(hatecheck_df["functionality"].isin(heldout_hate_funcs + heldout_nonhate_funcs) |
                                hatecheck_df["target_ident"].isin(heldout_idents))].copy()

In [ ]:
assert ((hatecheck_seen["functionality"].isin(heldout_hate_funcs + heldout_nonhate_funcs)).sum() + 
         hatecheck_seen["target_ident"].isin(heldout_idents).sum()) == 0

In [ ]:
hatecheck_unseen = hatecheck_df[(hatecheck_df["functionality"].isin(heldout_hate_funcs + heldout_nonhate_funcs) |
                                hatecheck_df["target_ident"].isin(heldout_idents))].copy()

In [ ]:
hatecheck_unseen

In [ ]:
assert len(hatecheck_seen) + len(hatecheck_unseen) == len(hatecheck_df)

In [ ]:
df_train, df_valtest = train_test_split(hatecheck_seen, test_size=0.2, stratify=hatecheck_seen.label_gold, random_state=42)
df_valtest = pd.concat([df_valtest, hatecheck_unseen])
df_val, df_test = train_test_split(df_valtest, test_size=0.5, stratify=df_valtest.label_gold, random_state=42)

In [ ]:
df_train.label_gold.value_counts(), df_val.label_gold.value_counts(), df_test.label_gold.value_counts()

In [ ]:
n_test_cases = len(df_val) + len(df_test); n_test_cases, n_test_cases/len(hatecheck_df)

In [ ]:
assert len(df_train) + len(df_val) + len(df_test) == len(hatecheck_df)

In [ ]:
train_texts = df_train.test_case.astype("string").tolist()
val_texts = df_val.test_case.astype("string").tolist()
test_texts = df_test.test_case.astype("string").tolist()
    
train_labels = df_train.label_gold.tolist()
val_labels = df_val.label_gold.tolist()
test_labels = df_test.label_gold.tolist()

In [ ]:
class_weights = compute_class_weight('balanced', classes = np.unique(train_labels), y = train_labels); class_weights

In [ ]:
# tokenizer = BertTokenizerFast.from_pretrained("BERT_davidson2017_weighted/Final")
if DEV:
    tokenizer = BertTokenizerFast.from_pretrained('prajjwal1/bert-tiny')
else:
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
num_added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[USER]','[EMOJI]','[URL]']}); num_added_toks

In [ ]:
# Tokenize text series
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
train_dataset = HateDataset(train_encodings, train_labels)
val_dataset = HateDataset(val_encodings, val_labels)
test_dataset = HateDataset(test_encodings, test_labels)

In [ ]:
torch.save(train_dataset, "data/unseenSplit_train_hatecheck.pt")
torch.save(val_dataset, "data/unseenSplit_val_hatecheck.pt")
torch.save(test_dataset, "data/unseenSplit_test_hatecheck.pt")

In [ ]:
# check CUDA availability
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())
    print(torch.cuda.device_count(), 'GPUs')
else:
    print("Oops! No GPU found.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Define training arguments

training_args = TrainingArguments(
    save_steps = 2500,
    output_dir="./models/BERT_hatecheck_weighted/checkpoints", # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    evaluation_strategy = 'epoch',
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate = 5e-5,
    seed = 123
)

In [ ]:
# define Trainers with weighted loss

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        weighted_loss = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weights)).to(device)
        return (weighted_loss(logits,labels), outputs) if return_outputs else weighted_loss(logits, labels)

In [ ]:
# define explicit model initialisation to allow for hyperparameter search
def model_init():
    if DEV:
        model = BertForSequenceClassification.from_pretrained("prajjwal1/bert-tiny")
    else:
        model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
    # resize to match tokenizer length with special tokens added above
    model.resize_token_embeddings(len(tokenizer))
    return model

In [ ]:
# Instantiate weighted trainer objects

trainer = WeightedTrainer(                       
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
    model_init = model_init
)

In [ ]:
# perform hyperparameter tuning using grid search
def custom_hp_space(trial):
    if DEV:
        return {
        "learning_rate": tune.grid_search([2e-5, 3e-5, 5e-5]),
        "num_train_epochs": tune.grid_search([2, 3, 4]),
        "per_device_train_batch_size": tune.grid_search([4,8])
    }
    else:
        return {
            "learning_rate": tune.grid_search([2e-5, 3e-5, 5e-5]),
            "num_train_epochs": tune.grid_search([2, 3, 4]),
            "per_device_train_batch_size": tune.grid_search([16,32])
        }

In [ ]:
best_run = trainer.hyperparameter_search(
    backend = 'ray',
    hp_space = custom_hp_space,
    direction = 'minimize',
    n_trials = 1,
    progress_reporter = tune.JupyterNotebookReporter(overwrite=True)
)

In [ ]:
print(best_run)

In [ ]:
for n, v in best_run.hyperparameters.items():
        setattr(trainer.args, n, v)

In [ ]:
trainer.train()

In [ ]:
if DEV:
    trainer.save_model('./models/BERT_hateCheck_weighted_unseen/dev')
    tokenizer.save_pretrained('./models/BERT_hateCheck_weighted_unseen/dev')
else:
    trainer.save_model('./models/BERT_hateCheck_weighted_unseen/final')
    tokenizer.save_pretrained('./models/BERT_hateCheck_weighted_unseen/final')

In [ ]:
# load fine-tuned model

if DEV:
    model = BertForSequenceClassification.from_pretrained('./models/BERT_hateCheck_weighted_unseen/dev')
else:
    model = BertForSequenceClassification.from_pretrained('./models/BERT_hateCheck_weighted_unseen/final') 

In [ ]:
if DEV:
    trainer = Trainer(
        model=model,         
        args=TrainingArguments(
            output_dir=('./models/BERT_hateCheck_weighted_unseen/devTest'),
            per_device_eval_batch_size = 64)
    )
else:
    trainer = Trainer(
        model=model,         
        args=TrainingArguments(
            output_dir=('./models/BERT_hateCheck_weighted_unseen/test'),
            per_device_eval_batch_size = 64)
    )

In [ ]:
train_results, train_preds = get_results(trainer, train_dataset)

In [ ]:
val_results, val_preds = get_results(trainer, val_dataset)

In [ ]:
test_results, test_preds = get_results(trainer, test_dataset)

In [ ]:
df_train["preds"], df_train["split"] = train_preds, "train"
df_val["preds"], df_val["split"] = val_preds, "val"
df_test["preds"], df_test["split"] = test_preds, "test"

In [ ]:
preds_df = pd.concat([df_train, df_val, df_test], axis= 0)

In [ ]:
preds_df["preds"].replace({1: 'hateful', 0: 'non-hateful'}, inplace=True)

In [ ]:
preds_df

In [ ]:
if DEV:
    preds_df.to_pickle('./results/hatecheck/resultsDEV_BERT_weighted_unseen.pkl')
else:
    preds_df.to_pickle('./results/hatecheck/results_BERT_weighted_unseen.pkl')

## Random stratified splitting

In [ ]:
df_train, df_valtest = train_test_split(hatecheck_df, test_size=n_test_cases, stratify=hatecheck_df.label_gold, random_state=123)
df_val, df_test = train_test_split(df_valtest, test_size=0.5, stratify=df_valtest.label_gold, random_state=123)

In [ ]:
df_train.label_gold.value_counts(), df_val.label_gold.value_counts(), df_test.label_gold.value_counts()

In [ ]:
train_texts = df_train.test_case.astype("string").tolist()
val_texts = df_val.test_case.astype("string").tolist()
test_texts = df_test.test_case.astype("string").tolist()
    
train_labels = df_train.label_gold.tolist()
val_labels = df_val.label_gold.tolist()
test_labels = df_test.label_gold.tolist()

In [ ]:
class_weights = compute_class_weight('balanced', classes = np.unique(train_labels), y = train_labels); class_weights

In [ ]:
# tokenizer = BertTokenizerFast.from_pretrained("BERT_davidson2017_weighted/Final")
if DEV:
    tokenizer = BertTokenizerFast.from_pretrained('prajjwal1/bert-tiny')
else:
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
num_added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[USER]','[EMOJI]','[URL]']}); num_added_toks

In [ ]:
# Tokenize text series
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
train_dataset = HateDataset(train_encodings, train_labels)
val_dataset = HateDataset(val_encodings, val_labels)
test_dataset = HateDataset(test_encodings, test_labels)

In [ ]:
torch.save(train_dataset, "data/random_train_hatecheck.pt")
torch.save(val_dataset, "data/random_val_hatecheck.pt")
torch.save(test_dataset, "data/random_test_hatecheck.pt")

In [ ]:
# Define training arguments

training_args = TrainingArguments(
    save_steps = 2500,
    output_dir="./models/BERT_hatecheck_weighted/checkpoints", # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    evaluation_strategy = 'epoch',
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate = 5e-5,
    seed = 123
)

In [ ]:
# Instantiate weighted trainer objects

trainer = WeightedTrainer(                       
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
    model_init = model_init
)

In [ ]:
# perform hyperparameter tuning using grid search
def custom_hp_space(trial):
    if DEV:
        return {
        "learning_rate": tune.grid_search([2e-5, 3e-5, 5e-5]),
        "num_train_epochs": tune.grid_search([2, 3, 4]),
        "per_device_train_batch_size": tune.grid_search([4,8])
    }
    else:
        return {
            "learning_rate": tune.grid_search([2e-5, 3e-5, 5e-5]),
            "num_train_epochs": tune.grid_search([2, 3, 4]),
            "per_device_train_batch_size": tune.grid_search([16,32])
        }

In [ ]:
best_run = trainer.hyperparameter_search(
    backend = 'ray',
    hp_space = custom_hp_space,
    direction = 'minimize',
    n_trials = 1,
    progress_reporter = tune.JupyterNotebookReporter(overwrite=True)
)

In [ ]:
print(best_run)

In [ ]:
for n, v in best_run.hyperparameters.items():
        setattr(trainer.args, n, v)

In [ ]:
trainer.train()

In [ ]:
if DEV:
    trainer.save_model('./models/BERT_hateCheck_weighted/dev')
    tokenizer.save_pretrained('./models/BERT_hateCheck_weighted/dev')
else:
    trainer.save_model('./models/BERT_hateCheck_weighted/final')
    tokenizer.save_pretrained('./models/BERT_hateCheck_weighted/final')

In [ ]:
# load fine-tuned model

if DEV:
    model = BertForSequenceClassification.from_pretrained('./models/BERT_hateCheck_weighted/dev')
else:
    model = BertForSequenceClassification.from_pretrained('./models/BERT_hateCheck_weighted/final') 

In [ ]:
if DEV:
    trainer = Trainer(
        model=model,         
        args=TrainingArguments(
            output_dir=('./models/BERT_hateCheck_weighted/devTest'),
            per_device_eval_batch_size = 64)
    )
else:
    trainer = Trainer(
        model=model,         
        args=TrainingArguments(
            output_dir=('./models/BERT_hateCheck_weighted/test'),
            per_device_eval_batch_size = 64)
    )

In [ ]:
train_results, train_preds = get_results(trainer, train_dataset)

In [ ]:
val_results, val_preds = get_results(trainer, val_dataset)

In [ ]:
test_results, test_preds = get_results(trainer, test_dataset)

In [ ]:
df_train["preds"], df_train["split"] = train_preds, "train"
df_val["preds"], df_val["split"] = val_preds, "val"
df_test["preds"], df_test["split"] = test_preds, "test"

In [ ]:
preds_df = pd.concat([df_train, df_val, df_test], axis= 0)

In [ ]:
preds_df["preds"].replace({1: 'hateful', 0: 'non-hateful'}, inplace=True)

In [ ]:
preds_df

In [ ]:
if DEV:
    preds_df.to_pickle('./results/hatecheck/resultsDEV_BERT_weighted.pkl')
else:
    preds_df.to_pickle('./results/hatecheck/results_BERT_weighted.pkl')

## Data augmentation

In [ ]:
df_train, df_valtest = train_test_split(hatecheck_df, test_size=n_test_cases, stratify=hatecheck_df.label_gold, random_state=123)
df_val, df_test = train_test_split(df_valtest, test_size=0.5, stratify=df_valtest.label_gold, random_state=123)

In [ ]:
df_train.label_gold.value_counts(), df_val.label_gold.value_counts(), df_test.label_gold.value_counts()

In [ ]:
training_data = pd.read_pickle('./hatecheck-experiments/Data/Clean Training Data/training_data_binary.pkl')

df_raw = {}

# write to dict
for dataset in training_data:
    df_raw[dataset] = training_data[dataset].copy()

In [ ]:
df_train_target = {}

for dataset in df_raw:
    df_train_target[dataset], _ = train_test_split(df_raw[dataset], test_size=0.2, stratify=df_raw[dataset].label, random_state=123)

In [ ]:
len(df_train), len(df_train_target["davidson2017"]), len(df_train_target["founta2018"])

In [ ]:
train_texts_with_davidson = df_train.test_case.astype("string").tolist() + df_train_target["davidson2017"].text.astype("string").tolist()
train_texts_with_founta = df_train.test_case.astype("string").tolist() + df_train_target["founta2018"].text.astype("string").tolist()
val_texts = df_val.test_case.astype("string").tolist()
test_texts = df_test.test_case.astype("string").tolist()
    
train_labels_with_davidson = df_train.label_gold.tolist() + df_train_target["davidson2017"].label.tolist()
train_labels_with_founta = df_train.label_gold.tolist() + df_train_target["founta2018"].label.tolist()
val_labels = df_val.label_gold.tolist()
test_labels = df_test.label_gold.tolist()

In [ ]:
# tokenizer = BertTokenizerFast.from_pretrained("BERT_davidson2017_weighted/Final")
if DEV:
    tokenizer = BertTokenizerFast.from_pretrained('prajjwal1/bert-tiny')
else:
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
num_added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[USER]','[EMOJI]','[URL]']}); num_added_toks

In [ ]:
# Tokenize text series
train_encodings_with_davidson = tokenizer(train_texts_with_davidson, truncation=True, padding=True)
train_encodings_with_founta = tokenizer(train_texts_with_founta, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
class_weights_davidson = compute_class_weight('balanced', classes = np.unique(train_labels_with_davidson), y = train_labels_with_davidson)
class_weights_founta = compute_class_weight('balanced', classes = np.unique(train_labels_with_founta), y = train_labels_with_founta)
class_weights_davidson, class_weights_founta

In [ ]:
train_dataset_with_davidson = HateDataset(train_encodings_with_davidson, train_labels_with_davidson)
train_dataset_with_founta = HateDataset(train_encodings_with_founta, train_labels_with_founta)
val_dataset = HateDataset(val_encodings, val_labels)
test_dataset = HateDataset(test_encodings, test_labels)

In [ ]:
torch.save(train_dataset_with_davidson, "data/random_train_hatecheck+davidson.pt")
torch.save(train_dataset_with_founta, "data/random_train_hatecheck+founta.pt")

In [ ]:
# Define training arguments

training_args = TrainingArguments(
    save_steps = 2500,
    output_dir="./models/BERT_hatecheck_weighted/checkpoints", # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    evaluation_strategy = 'epoch',
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate = 5e-5,
    seed = 123
)

In [ ]:
class WeightedTrainerDavidson(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        weighted_loss = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weights_davidson)).to(device)
        return (weighted_loss(logits,labels), outputs) if return_outputs else weighted_loss(logits, labels)

In [ ]:
class WeightedTrainerFounta(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        weighted_loss = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weights_founta)).to(device)
        return (weighted_loss(logits,labels), outputs) if return_outputs else weighted_loss(logits, labels)

In [ ]:
# Instantiate weighted trainer objects

trainer = WeightedTrainerDavidson(                       
    args=training_args,                  
    train_dataset=train_dataset_with_davidson,         
    eval_dataset=val_dataset,            
    model_init = model_init
)

In [ ]:
# perform hyperparameter tuning using grid search
def custom_hp_space(trial):
    if DEV:
        return {
        "learning_rate": tune.grid_search([2e-5, 3e-5, 5e-5]),
        "num_train_epochs": tune.grid_search([2, 3, 4]),
        "per_device_train_batch_size": tune.grid_search([4,8])
    }
    else:
        return {
            "learning_rate": tune.grid_search([2e-5, 3e-5, 5e-5]),
            "num_train_epochs": tune.grid_search([2, 3, 4]),
            "per_device_train_batch_size": tune.grid_search([16,32])
        }

In [ ]:
best_run = trainer.hyperparameter_search(
    backend = 'ray',
    hp_space = custom_hp_space,
    direction = 'minimize',
    n_trials = 1,
    progress_reporter = tune.JupyterNotebookReporter(overwrite=True)
)

In [ ]:
print(best_run)

In [ ]:
for n, v in best_run.hyperparameters.items():
        setattr(trainer.args, n, v)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('./models/BERT_hateCheck+davidson_weighted/final')
tokenizer.save_pretrained('./models/BERT_hateCheck+davidson_weighted/final')

In [ ]:
model = BertForSequenceClassification.from_pretrained('./models/BERT_hateCheck+davidson_weighted/final') 

In [ ]:
if DEV:
    trainer = Trainer(
        model=model,         
        args=TrainingArguments(
            output_dir=('./models/BERT_hateCheck_weighted/devTest'),
            per_device_eval_batch_size = 64)
    )
else:
    trainer = Trainer(
        model=model,         
        args=TrainingArguments(
            output_dir=('./models/BERT_hateCheck_weighted/test'),
            per_device_eval_batch_size = 64)
    )

In [ ]:
train_results, train_preds = get_results(trainer, train_dataset_with_davidson)

In [ ]:
val_results, val_preds = get_results(trainer, val_dataset)

In [ ]:
test_results, test_preds = get_results(trainer, test_dataset)

In [ ]:
df_train["preds"], df_train["split"] = train_preds[:len(df_train)], "train"
df_val["preds"], df_val["split"] = val_preds, "val"
df_test["preds"], df_test["split"] = test_preds, "test"

In [ ]:
preds_df = pd.concat([df_train, df_val, df_test], axis= 0)

In [ ]:
preds_df["preds"].replace({1: 'hateful', 0: 'non-hateful'}, inplace=True)

In [ ]:
preds_df

In [ ]:
preds_df.to_pickle('./results/hatecheck/results_BERT_hateCheck+davidson_weighted.pkl')

In [ ]:
# Instantiate weighted trainer objects

trainer = WeightedTrainerFounta(                       
    args=training_args,                  
    train_dataset=train_dataset_with_founta,         
    eval_dataset=val_dataset,            
    model_init = model_init
)

In [ ]:
# perform hyperparameter tuning using grid search
def custom_hp_space(trial):
    if DEV:
        return {
        "learning_rate": tune.grid_search([2e-5, 3e-5, 5e-5]),
        "num_train_epochs": tune.grid_search([2, 3, 4]),
        "per_device_train_batch_size": tune.grid_search([4,8])
    }
    else:
        return {
            "learning_rate": tune.grid_search([2e-5, 3e-5, 5e-5]),
            "num_train_epochs": tune.grid_search([2, 3, 4]),
            "per_device_train_batch_size": tune.grid_search([16,32])
        }

In [ ]:
best_run = trainer.hyperparameter_search(
    backend = 'ray',
    hp_space = custom_hp_space,
    direction = 'minimize',
    n_trials = 1,
    progress_reporter = tune.JupyterNotebookReporter(overwrite=True)
)

2e-05	4	16	4	5804.12	0.00772648	

In [ ]:
print(best_run)

In [ ]:
for n, v in best_run.hyperparameters.items():
        setattr(trainer.args, n, v)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('./models/BERT_hateCheck+founta_weighted/final')
tokenizer.save_pretrained('./models/BERT_hateCheck+founta_weighted/final')

In [ ]:
model = BertForSequenceClassification.from_pretrained('./models/BERT_hateCheck+founta_weighted/final') 

In [ ]:
if DEV:
    trainer = Trainer(
        model=model,         
        args=TrainingArguments(
            output_dir=('./models/BERT_hateCheck_weighted/devTest'),
            per_device_eval_batch_size = 64)
    )
else:
    trainer = Trainer(
        model=model,         
        args=TrainingArguments(
            output_dir=('./models/BERT_hateCheck_weighted/test'),
            per_device_eval_batch_size = 64)
    )

In [ ]:
train_results, train_preds = get_results(trainer, train_dataset_with_founta)

In [ ]:
val_results, val_preds = get_results(trainer, val_dataset)

In [ ]:
test_results, test_preds = get_results(trainer, test_dataset)

In [ ]:
df_train["preds"], df_train["split"] = train_preds[:len(df_train)], "train"
df_val["preds"], df_val["split"] = val_preds, "val"
df_test["preds"], df_test["split"] = test_preds, "test"

In [ ]:
preds_df = pd.concat([df_train, df_val, df_test], axis= 0)

In [ ]:
preds_df["preds"].replace({1: 'hateful', 0: 'non-hateful'}, inplace=True)

In [ ]:
preds_df

In [ ]:
preds_df.to_pickle('./results/hatecheck/results_BERT_hateCheck+founta_weighted.pkl')